In [1]:
import numpy as np
from szar.counts import ClusterCosmology,Halo_MF
from szar.szproperties import SZ_Cluster_Model
from configparser import SafeConfigParser
from orphics.tools.io import dictFromSection,listFromConfig
import cPickle as pickle

home = '/Users/nab/Repos/SZ_filter/'

!pwd

iniFile = home+'input/pipeline.ini'
expName = 'CCATP-propv2'
gridName = 'grid-owl2'
cal = 'owl2'

Config = SafeConfigParser()
Config.optionxform=str
Config.read(iniFile)

fparams = {}
for (key, val) in Config.items('params'):
    if ',' in val:
        param, step = val.split(',')
        fparams[key] = float(param)
    else:
        fparams[key] = float(val)

bigDataDir = Config.get('general','bigDataDirectory')
clttfile = Config.get('general','clttfile')
constDict = dictFromSection(Config,'constants')
clusterDict = dictFromSection(Config,'cluster_params')
beam = listFromConfig(Config,expName,'beams')
noise = listFromConfig(Config,expName,'noises')
freq = listFromConfig(Config,expName,'freqs')
lknee = listFromConfig(Config,expName,'lknee')[0]
alpha = listFromConfig(Config,expName,'alpha')[0]

calFile = bigDataDir+"lensgrid_grid-"+cal+"_"+cal+".pkl"

version = Config.get('general','version')

mexp_edges, z_edges, lndM = pickle.load(open(calFile,"rb"))

mgrid,zgrid,siggrid = pickle.load(open(bigDataDir+"szgrid_"+expName+"_"+gridName+ "_v" + version+".pkl",'rb'))

assert np.all(mgrid==mexp_edges)
assert np.all(z_edges==zgrid)

#print lndM

cc = ClusterCosmology(fparams,constDict,clTTFixFile=clttfile)
HMF = Halo_MF(cc,mgrid,zgrid)

qs = listFromConfig(Config,'general','qbins')
qspacing = Config.get('general','qbins_spacing')
if qspacing=="log":
    qbin_edges = np.logspace(np.log10(qs[0]),np.log10(qs[1]),int(qs[2])+1)
elif qspacing=="linear":
    qbin_edges = np.linspace(qs[0],qs[1],int(qs[2])+1)
else:
    raise ValueError
    
SZProp = SZ_Cluster_Model(cc,clusterDict,rms_noises = noise,fwhms=beam,freqs=freq,lknee=lknee,alpha=alpha)

Mwl = 10**HMF.Mexp
z_arr = HMF.zarr
M_arr =  np.outer(HMF.M,np.ones([len(z_arr)]))

#if HMF.sigN is None: HMF.updateSigN(SZProp)
#sigN = HMF.sigN
#np.savetxt(home+'/tests/sigN_arr_save.txt',sigN)

sigN = np.loadtxt(home+'/tests/sigN_arr_save.txt')

#SZProp = SZ_Cluster_Model(cc,clusterDict,rms_noises = noise,fwhms=beam,freqs=freq,lknee=lknee,alpha=alpha)

q_arr = (qbin_edges[1:]+qbin_edges[:-1])/2.

i = 5
kk = 2
jj = 4

blah_orig = SZProp.P_of_qn(SZProp.lnY,M_arr[:,i],z_arr[i],sigN[:,i],q_arr[kk])
blah_mwl = SZProp.Mwl_prob (Mwl[jj],M_arr[:,i],lndM[:,i])
blah = SZProp.P_of_qn_corr(SZProp.lnY,M_arr[:,i],z_arr[i],sigN[:,i],q_arr[kk],Mwl[jj],lndM[:,i])

#print blah_orig
#print blah_mwl
print blah_orig * blah_mwl
print blah

#dN_dmqz_corr = HMF.N_of_mqz_SZ_corr(lndM,qbin_edges,SZProp)                                                                  
#dN_dmqz = HMF.N_of_mqz_SZ(lndM,qbin_edges,SZProp)

/Users/nab/Repos/orphics/orphics/tools/io.py:3: UserWarning: WARNING: This module is deprecated. Most of its contents have moved to orphics.io. If you do not find the function you require there, please raise an issue.
  warnings.warn("WARNING: This module is deprecated. Most of its contents have moved to orphics.io. If you do not find the function you require there, please raise an issue.")
/Users/nab/Repos/orphics/orphics/tools/stats.py:3: UserWarning: WARNING: This module is deprecated. Most of its contents have moved to orphics.stats. If you do not find the function you require there, please raise an issue.
  warnings.warn("WARNING: This module is deprecated. Most of its contents have moved to orphics.stats. If you do not find the function you require there, please raise an issue.")
/Users/nab/Repos/orphics/orphics/analysis/flatMaps.py:3: UserWarning: WARNING: This module is deprecated. Most of its contents have moved to orphics.maps. If you do not find the function you require ther

/Users/nab/Repos/SZ_filter
((222, 130), (), (222,))
[[  7.07813258e-09   7.14425702e-09   7.21803433e-09 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  7.07443593e-09   7.14013461e-09   7.21343277e-09 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  7.07074030e-09   7.13601355e-09   7.20883300e-09 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 ..., 
 [  6.53453080e-09   6.54046133e-09   6.54702165e-09 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  6.53367034e-09   6.53950954e-09   6.54596874e-09 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  6.53282105e-09   6.53857011e-09   6.54492953e-09 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]]
No corr
((222, 130), (222, 130), (130,), (129,))
size 2
((222, 130), (), (222,), (222,))
[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1. 

        [  0.00000000e+00,   6.31518872e-28]]))
('gauss2D', (130,), (2, 130), (2, 2), ())
(matrix([[  1.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   1.62004964e+27]]), matrix([[  1.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   6.17265037e-28]]))
('gauss2D', (130,), (2, 130), (2, 2), ())
(matrix([[  1.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   1.65945828e+27]]), matrix([[  1.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   6.02606291e-28]]))
('gauss2D', (130,), (2, 130), (2, 2), ())
(matrix([[  1.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   1.70242475e+27]]), matrix([[  1.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   5.87397476e-28]]))
('gauss2D', (130,), (2, 130), (2, 2), ())
(matrix([[  1.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   1.74893327e+27]]), matrix([[  1.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   5.71777105e-28]]))
('gauss2D', (130,), (2, 130)

        [  0.00000000e+00,   2.16749836e-28]]))
('gauss2D', (130,), (2, 130), (2, 2), ())
(matrix([[  1.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   4.68034341e+27]]), matrix([[  1.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   2.13659535e-28]]))
('gauss2D', (130,), (2, 130), (2, 2), ())
(matrix([[  1.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   4.75615277e+27]]), matrix([[  1.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   2.10253970e-28]]))
('gauss2D', (130,), (2, 130), (2, 2), ())
(matrix([[  1.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   4.84117999e+27]]), matrix([[  1.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   2.06561211e-28]]))
('gauss2D', (130,), (2, 130), (2, 2), ())
(matrix([[  1.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   4.93536764e+27]]), matrix([[  1.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   2.02619151e-28]]))
('gauss2D', (130,), (2, 130)

        [  0.00000000e+00,   1.12974917e-28]]))
('gauss2D', (130,), (2, 130), (2, 2), ())
(matrix([[  1.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   8.99048005e+27]]), matrix([[  1.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   1.11228766e-28]]))
('gauss2D', (130,), (2, 130), (2, 2), ())
(matrix([[  1.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   9.16041564e+27]]), matrix([[  1.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   1.09165352e-28]]))
('gauss2D', (130,), (2, 130), (2, 2), ())
(matrix([[  1.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   9.36189059e+27]]), matrix([[  1.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   1.06816031e-28]]))
('gauss2D', (130,), (2, 130), (2, 2), ())
(matrix([[  1.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   9.59466185e+27]]), matrix([[  1.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   1.04224622e-28]]))
('gauss2D', (130,), (2, 130)

        [  0.00000000e+00,   4.86103545e-29]]))
('gauss2D', (130,), (2, 130), (2, 2), ())
(matrix([[  1.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   2.09711022e+28]]), matrix([[  1.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   4.76846657e-29]]))
('gauss2D', (130,), (2, 130), (2, 2), ())
(matrix([[  1.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   2.13480650e+28]]), matrix([[  1.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   4.68426529e-29]]))
('gauss2D', (130,), (2, 130), (2, 2), ())
(matrix([[  1.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   2.17047306e+28]]), matrix([[  1.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   4.60729054e-29]]))
('gauss2D', (130,), (2, 130), (2, 2), ())
(matrix([[  1.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   2.20444785e+28]]), matrix([[  1.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   4.53628331e-29]]))
('gauss2D', (130,), (2, 130)

        [  0.00000000e+00,   1.86813446e-29]]))
('gauss2D', (130,), (2, 130), (2, 2), ())
(matrix([[  1.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   5.60520976e+28]]), matrix([[  1.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   1.78405455e-29]]))
('gauss2D', (130,), (2, 130), (2, 2), ())
(222, 130)
[[  2.67039875e-13   2.66900825e-13   2.66747194e-13 ...,   1.75504488e-07
    2.14412104e-07   2.61939284e-07]
 [  2.58621900e-13   2.58487994e-13   2.58340047e-13 ...,   1.68007777e-07
    2.05253733e-07   2.50751161e-07]
 [  2.50935713e-13   2.50806538e-13   2.50663816e-13 ...,   1.61104588e-07
    1.96820442e-07   2.40448818e-07]
 ..., 
 [  3.57267929e-13   3.57266653e-13   3.57265244e-13 ...,   1.53095811e-10
    1.87374690e-10   2.29291613e-10]
 [  3.50470969e-13   3.50469740e-13   3.50468381e-13 ...,   1.45209086e-10
    1.77726461e-10   2.17490102e-10]
 [  3.42594437e-13   3.42593254e-13   3.42591947e-13 ...,   1.37547549e-10
    1.68353587e-10   2.060

/Users/nab/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:17: DeprecationWarning: The SafeConfigParser class has been renamed to ConfigParser in Python 3.2. This alias will be removed in future versions. Use ConfigParser directly instead.
/Users/nab/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:19: DeprecationWarning: You passed a bytestring as `filenames`. This will not work on Python 3. Use `cp.read_file()` or switch to using Unicode strings across the board.
